In [24]:
import os
cwd = os.getcwd() # Lugar de trabajo, se usará como variable global

In [25]:
import random
random.choice(list((23, 21)))

23

In [26]:
def generador_chunks():
    i = 0 # Numero que indica la siguiente carpeta a usar
    dp = 0 # Numero que indica el siguiente nombre de carpeta a usar en la carpeta par 
    di = 0 # Numero que indica el siguiente nombre de carpeta a usar en la carpeta impar
    while True:
        # Vemos a que carpeta ir
        if i % 2 == 0:
            nwd = cwd + os.sep + "par"
            j = dp # Vemos cual nombre de carpeta en par toca
            dp += 1
        else:
            nwd = cwd + os.sep + "impar"
            j = di # Vemos cual nombre de carpeta en impar toca
            di += 1

        for k, file in enumerate(os.listdir(nwd)):
            # Se ve si es que la carpeta es la que corresponde
            if j == k:
                yield int(file) # Se devuelve el nombre de la carpeta en forma numérica
                break
        
        # Esta sección reinicia las carpetas si es que ya se recorrieron las 3 en par o impar
        if dp > 2:
            dp = 0

        if di > 2:
            di = 0

        i += 1

In [27]:
# Función que decodifica un byte cualquiera
def decodificar_byte(byte):
    numero = 255 - byte  # Se refleja el byte
    numero = bin(numero) # Se transforma a binario
    new = ""
    for binary in numero[2:].zfill(8):
        new += "1" if binary == "0" else "0" # Se cambian cada numero del binario
    byte_final = int(new, 2) # Se vuelve a formar a decimal
    return byte_final

In [28]:
def algoritmo_decodificador(chunk):
    byte_sub_chunk = 0
    j = sum(int(d) for d in str(len(chunk))) # Suma de los dígitos
    byte_corte = None
    # Vemos si hay un byte de corte
    if j > 15:
        byte_corte = 100
    elif j < 15:
        byte_corte = 50
    nuevo_chunk = bytearray() # Arreglo del nuevo chunk a decodificar
    sub_chunk = bytearray() # Arreglo por sección de chunk si es que hay un corte
    for byte in chunk:
        # Vemos si se llega al byte de corte
        if byte_sub_chunk == byte_corte: 
            sub_chunk.append(byte) # Se agrega el byte sin procesar
            sub_chunk.reverse() # Se invierte el arreglo
            nuevo_chunk.extend(sub_chunk) # Se arreglan al arreglo final
            
            byte_sub_chunk = 0 # Reiniciamos el punto actual para el corte
            sub_chunk.clear() # Vaciamos el arreglo de la sección del chunk
            continue
        sub_chunk.append(decodificar_byte(byte)) # Procesamos el byte y lo agregamos al arreglo de sub_chunk
        byte_sub_chunk += 1
    nuevo_chunk.extend(sub_chunk) # Agregamos el arreglo que no se invirtió (Notemos que además agrega el arreglo que no requería inversiones con j  == 15)
    return nuevo_chunk

In [29]:
# ------------------ #
# PROGRAMA PRINCIPAL #
# ------------------ #

# Extraemos los bytes de la imagen codificada
with open(cwd + os.sep + "Curriculum.banana", mode="rb") as imagen:
    imagen_codificada = imagen.read()

# Aquí guardaremos los bytes decodificados
decoded_array = bytearray()
f_chunks = generador_chunks()  # Generador de tamaños 

pre_size = 0 # Punto de partida del chunk
max_size = len(imagen_codificada) - 27 # Aquí indicamos que los últimos 27 bytes no se tocan

while pre_size < max_size:
    size_chunk = next(f_chunks) # obtenemos el tamaño del chunk
    chunk = imagen_codificada[pre_size: pre_size + size_chunk] # sacamos el chunk
    decoded_array.extend(algoritmo_decodificador(chunk)) # Se agrega el chunk decodificado al arreglo final
    pre_size += size_chunk # Movemos el punto de partida del chunk
    
decoded_array.extend(imagen_codificada[pre_size: len(imagen_codificada)]) # Agregamos los últimos 27 bytes

# Guardamos los bytes en un archivo nuevo
with open(cwd + os.sep + "curriculum_vitae.jpg", mode="wb") as image:
    image.write(decoded_array)